In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+ '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
print(questions[:5], expected[:5])

['70+9   ', '3+21   ', '8+7    ', '8+91   ', '30+80  '] ['79  ', '24  ', '15  ', '99  ', '110 ']


# Processing

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(72000, 7, 12)
(72000, 4, 12)
Validation Data:
(8000, 7, 12)
(8000, 4, 12)


In [13]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False  True False False False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False False  True
   False]
  [False False False False False False False False False False False
    True]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False False False False False False False False  True False False
   False]
  [False False False False False False Fal

# Build Model

In [14]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [15]:
for iteration in range(40):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 8s 104us/step - loss: 1.8724 - acc: 0.3210 - val_loss: 1.7844 - val_acc: 0.3468
Q 395+7   T 402  ☒ 100 
Q 811+9   T 820  ☒ 100 
Q 448+31  T 479  ☒ 500 
Q 21+793  T 814  ☒ 200 
Q 101+658 T 759  ☒ 100 
Q 967+166 T 1133 ☒ 1000
Q 946+332 T 1278 ☒ 100 
Q 878+92  T 970  ☒ 100 
Q 537+943 T 1480 ☒ 1000
Q 11+878  T 889  ☒ 100 

--------------------------------------------------
Iteration 1
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 6s 90us/step - loss: 1.6335 - acc: 0.3887 - val_loss: 1.4982 - val_acc: 0.4430
Q 35+16   T 51   ☒ 11  
Q 27+206  T 233  ☒ 230 
Q 615+64  T 679  ☒ 650 
Q 769+550 T 1319 ☒ 1317
Q 530+605 T 1135 ☒ 1177
Q 507+45  T 552  ☒ 550 
Q 6+888   T 894  ☒ 987 
Q 303+954 T 1257 ☒ 1207
Q 745+71  T 816  ☒ 800 
Q 50+552  T 602  ☒ 659 

--------------------

72000/72000 [==============================] - 7s 90us/step - loss: 0.0553 - acc: 0.9879 - val_loss: 0.0458 - val_acc: 0.9903
Q 61+918  T 979  ☑ 979 
Q 97+252  T 349  ☑ 349 
Q 92+319  T 411  ☑ 411 
Q 78+135  T 213  ☑ 213 
Q 79+195  T 274  ☑ 274 
Q 417+14  T 431  ☑ 431 
Q 846+3   T 849  ☑ 849 
Q 677+74  T 751  ☑ 751 
Q 631+20  T 651  ☑ 651 
Q 445+131 T 576  ☑ 576 

--------------------------------------------------
Iteration 15
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 7s 94us/step - loss: 0.0518 - acc: 0.9880 - val_loss: 0.0366 - val_acc: 0.9927
Q 68+802  T 870  ☑ 870 
Q 590+95  T 685  ☑ 685 
Q 400+499 T 899  ☑ 899 
Q 25+16   T 41   ☑ 41  
Q 52+675  T 727  ☑ 727 
Q 9+472   T 481  ☑ 481 
Q 839+72  T 911  ☑ 911 
Q 52+263  T 315  ☑ 315 
Q 524+92  T 616  ☑ 616 
Q 667+24  T 691  ☑ 691 

--------------------------------------------------
Iteration 16
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [=======

72000/72000 [==============================] - 7s 91us/step - loss: 0.0073 - acc: 0.9988 - val_loss: 0.0104 - val_acc: 0.9971
Q 64+154  T 218  ☑ 218 
Q 62+560  T 622  ☑ 622 
Q 85+535  T 620  ☑ 620 
Q 113+566 T 679  ☑ 679 
Q 5+66    T 71   ☑ 71  
Q 36+899  T 935  ☑ 935 
Q 80+38   T 118  ☑ 118 
Q 445+80  T 525  ☑ 525 
Q 56+738  T 794  ☑ 794 
Q 137+39  T 176  ☑ 176 

--------------------------------------------------
Iteration 29
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 6s 90us/step - loss: 0.0277 - acc: 0.9921 - val_loss: 0.0413 - val_acc: 0.9867
Q 245+27  T 272  ☑ 272 
Q 52+319  T 371  ☑ 371 
Q 50+275  T 325  ☑ 325 
Q 407+19  T 426  ☑ 426 
Q 677+37  T 714  ☑ 714 
Q 653+7   T 660  ☑ 660 
Q 370+76  T 446  ☑ 446 
Q 693+313 T 1006 ☒ 1005
Q 686+44  T 730  ☑ 730 
Q 810+139 T 949  ☑ 949 

--------------------------------------------------
Iteration 30
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [=======

In [ ]:
print("MSG : Prediction")
test_x = ["555+275", "860+7  ", "340+29 "]
test_y = ["830 ", "867 ", "369 "]
x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
for j, (i, c) in enumerate(zip(test_x, test_y)):
    x[j] = ctable.encode(i, MAXLEN)
    y[j] = ctable.encode(c, DIGITS + 1)

# Validation

In [19]:
right = 0
preds = model.predict_classes(x_val, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(x_val[i])
    correct = ctable.decode(y_val[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

Q 13+51   T 64   ☑ 64  
Q 50+2    T 52   ☑ 52  
Q 836+35  T 871  ☑ 871 
Q 624+92  T 716  ☑ 716 
Q 31+498  T 529  ☑ 529 
Q 716+83  T 799  ☑ 799 
Q 582+846 T 1428 ☑ 1428
Q 520+159 T 679  ☑ 679 
Q 208+638 T 846  ☑ 846 
Q 710+393 T 1103 ☑ 1103
Q 279+849 T 1128 ☑ 1128
Q 2+169   T 171  ☑ 171 
Q 71+736  T 807  ☑ 807 
Q 470+38  T 508  ☑ 508 
Q 862+41  T 903  ☑ 903 
Q 61+30   T 91   ☑ 91  
Q 43+960  T 1003 ☑ 1003
Q 772+65  T 837  ☑ 837 
Q 284+659 T 943  ☑ 943 
Q 2+911   T 913  ☑ 913 
Q 599+233 T 832  ☑ 832 
Q 329+0   T 329  ☑ 329 
Q 80+691  T 771  ☑ 771 
Q 78+52   T 130  ☑ 130 
Q 55+222  T 277  ☑ 277 
Q 99+12   T 111  ☑ 111 
Q 448+889 T 1337 ☑ 1337
Q 883+43  T 926  ☑ 926 
Q 31+257  T 288  ☑ 288 
Q 9+342   T 351  ☑ 351 
Q 46+98   T 144  ☑ 144 
Q 683+665 T 1348 ☑ 1348
Q 76+528  T 604  ☑ 604 
Q 84+98   T 182  ☑ 182 
Q 574+85  T 659  ☑ 659 
Q 828+791 T 1619 ☑ 1619
Q 115+6   T 121  ☑ 121 
Q 45+208  T 253  ☑ 253 
Q 734+683 T 1417 ☑ 1417
Q 89+731  T 820  ☑ 820 
Q 22+812  T 834  ☑ 834 
Q 83+126  T 209 

Q 606+48  T 654  ☑ 654 
Q 871+219 T 1090 ☑ 1090
Q 884+37  T 921  ☑ 921 
Q 839+750 T 1589 ☑ 1589
Q 67+31   T 98   ☑ 98  
Q 21+54   T 75   ☑ 75  
Q 467+42  T 509  ☑ 509 
Q 986+5   T 991  ☑ 991 
Q 427+79  T 506  ☑ 506 
Q 60+523  T 583  ☑ 583 
Q 16+920  T 936  ☑ 936 
Q 42+67   T 109  ☑ 109 
Q 427+62  T 489  ☑ 489 
Q 26+84   T 110  ☑ 110 
Q 9+171   T 180  ☑ 180 
Q 87+228  T 315  ☑ 315 
Q 150+58  T 208  ☑ 208 
Q 172+58  T 230  ☑ 230 
Q 572+122 T 694  ☑ 694 
Q 608+226 T 834  ☑ 834 
Q 70+248  T 318  ☑ 318 
Q 115+84  T 199  ☑ 199 
Q 474+33  T 507  ☑ 507 
Q 6+360   T 366  ☑ 366 
Q 617+995 T 1612 ☑ 1612
Q 616+900 T 1516 ☑ 1516
Q 1+808   T 809  ☑ 809 
Q 157+528 T 685  ☑ 685 
Q 70+26   T 96   ☑ 96  
Q 5+827   T 832  ☑ 832 
Q 38+427  T 465  ☑ 465 
Q 76+581  T 657  ☑ 657 
Q 205+19  T 224  ☑ 224 
Q 378+999 T 1377 ☑ 1377
Q 979+62  T 1041 ☑ 1041
Q 710+74  T 784  ☑ 784 
Q 75+961  T 1036 ☑ 1036
Q 88+951  T 1039 ☑ 1039
Q 632+613 T 1245 ☑ 1245
Q 7+845   T 852  ☑ 852 
Q 88+98   T 186  ☑ 186 
Q 62+728  T 790 

Q 227+39  T 266  ☑ 266 
Q 813+33  T 846  ☑ 846 
Q 587+299 T 886  ☑ 886 
Q 826+95  T 921  ☑ 921 
Q 0+243   T 243  ☑ 243 
Q 704+24  T 728  ☑ 728 
Q 848+180 T 1028 ☑ 1028
Q 898+40  T 938  ☑ 938 
Q 303+81  T 384  ☑ 384 
Q 639+90  T 729  ☑ 729 
Q 1+506   T 507  ☑ 507 
Q 793+14  T 807  ☑ 807 
Q 2+690   T 692  ☑ 692 
Q 194+4   T 198  ☑ 198 
Q 31+450  T 481  ☑ 481 
Q 831+36  T 867  ☑ 867 
Q 809+548 T 1357 ☑ 1357
Q 101+41  T 142  ☑ 142 
Q 79+270  T 349  ☑ 349 
Q 933+1   T 934  ☑ 934 
Q 57+141  T 198  ☑ 198 
Q 351+25  T 376  ☑ 376 
Q 525+523 T 1048 ☑ 1048
Q 306+953 T 1259 ☑ 1259
Q 82+68   T 150  ☑ 150 
Q 339+250 T 589  ☑ 589 
Q 718+37  T 755  ☑ 755 
Q 42+801  T 843  ☑ 843 
Q 24+482  T 506  ☑ 506 
Q 53+981  T 1034 ☑ 1034
Q 828+69  T 897  ☑ 897 
Q 84+154  T 238  ☑ 238 
Q 89+885  T 974  ☑ 974 
Q 48+23   T 71   ☑ 71  
Q 5+303   T 308  ☑ 308 
Q 563+206 T 769  ☑ 769 
Q 64+255  T 319  ☑ 319 
Q 644+767 T 1411 ☑ 1411
Q 55+524  T 579  ☑ 579 
Q 11+360  T 371  ☑ 371 
Q 6+23    T 29   ☑ 29  
Q 68+140  T 208 

Q 507+538 T 1045 ☑ 1045
Q 540+878 T 1418 ☑ 1418
Q 288+615 T 903  ☑ 903 
Q 847+271 T 1118 ☑ 1118
Q 944+981 T 1925 ☑ 1925
Q 138+50  T 188  ☑ 188 
Q 510+15  T 525  ☑ 525 
Q 131+70  T 201  ☑ 201 
Q 14+877  T 891  ☑ 891 
Q 910+871 T 1781 ☑ 1781
Q 875+6   T 881  ☑ 881 
Q 820+17  T 837  ☑ 837 
Q 129+188 T 317  ☑ 317 
Q 12+186  T 198  ☑ 198 
Q 27+132  T 159  ☑ 159 
Q 853+51  T 904  ☑ 904 
Q 49+617  T 666  ☑ 666 
Q 785+42  T 827  ☑ 827 
Q 916+605 T 1521 ☑ 1521
Q 46+718  T 764  ☑ 764 
Q 231+47  T 278  ☑ 278 
Q 605+250 T 855  ☑ 855 
Q 6+189   T 195  ☑ 195 
Q 597+8   T 605  ☑ 605 
Q 764+23  T 787  ☑ 787 
Q 36+795  T 831  ☑ 831 
Q 831+211 T 1042 ☑ 1042
Q 790+412 T 1202 ☑ 1202
Q 41+632  T 673  ☑ 673 
Q 764+697 T 1461 ☑ 1461
Q 406+804 T 1210 ☑ 1210
Q 29+205  T 234  ☑ 234 
Q 884+517 T 1401 ☑ 1401
Q 523+67  T 590  ☑ 590 
Q 739+41  T 780  ☑ 780 
Q 453+808 T 1261 ☑ 1261
Q 500+112 T 612  ☑ 612 
Q 152+37  T 189  ☑ 189 
Q 193+575 T 768  ☑ 768 
Q 6+90    T 96   ☑ 96  
Q 10+183  T 193  ☑ 193 
Q 887+55  T 942 

Q 955+68  T 1023 ☑ 1023
Q 173+756 T 929  ☑ 929 
Q 481+44  T 525  ☑ 525 
Q 50+695  T 745  ☑ 745 
Q 883+9   T 892  ☑ 892 
Q 699+506 T 1205 ☑ 1205
Q 3+63    T 66   ☑ 66  
Q 657+540 T 1197 ☑ 1197
Q 269+89  T 358  ☑ 358 
Q 43+990  T 1033 ☑ 1033
Q 103+7   T 110  ☑ 110 
Q 574+42  T 616  ☑ 616 
Q 570+210 T 780  ☑ 780 
Q 896+511 T 1407 ☑ 1407
Q 928+54  T 982  ☑ 982 
Q 5+974   T 979  ☑ 979 
Q 9+932   T 941  ☑ 941 
Q 421+220 T 641  ☑ 641 
Q 422+4   T 426  ☑ 426 
Q 929+23  T 952  ☑ 952 
Q 953+64  T 1017 ☑ 1017
Q 337+95  T 432  ☑ 432 
Q 628+1   T 629  ☑ 629 
Q 68+651  T 719  ☑ 719 
Q 8+438   T 446  ☑ 446 
Q 983+47  T 1030 ☑ 1030
Q 21+610  T 631  ☑ 631 
Q 949+67  T 1016 ☑ 1016
Q 84+805  T 889  ☑ 889 
Q 452+859 T 1311 ☑ 1311
Q 90+341  T 431  ☑ 431 
Q 321+423 T 744  ☑ 744 
Q 92+905  T 997  ☑ 997 
Q 279+162 T 441  ☑ 441 
Q 37+466  T 503  ☑ 503 
Q 57+422  T 479  ☑ 479 
Q 579+50  T 629  ☑ 629 
Q 22+856  T 878  ☑ 878 
Q 953+90  T 1043 ☑ 1043
Q 400+654 T 1054 ☑ 1054
Q 808+0   T 808  ☑ 808 
Q 359+94  T 453 

...

Q 0+296   T 296  ☑ 296 
Q 129+73  T 202  ☑ 202 
Q 5+337   T 342  ☑ 342 
Q 805+2   T 807  ☑ 807 
Q 5+681   T 686  ☑ 686 
Q 959+351 T 1310 ☑ 1310
Q 20+48   T 68   ☑ 68  
Q 425+746 T 1171 ☑ 1171
Q 88+915  T 1003 ☑ 1003
Q 611+66  T 677  ☑ 677 
Q 226+61  T 287  ☑ 287 
Q 232+77  T 309  ☑ 309 
Q 0+285   T 285  ☑ 285 
Q 609+869 T 1478 ☑ 1478
Q 980+39  T 1019 ☑ 1019
Q 3+407   T 410  ☑ 410 
Q 44+349  T 393  ☑ 393 
Q 383+257 T 640  ☑ 640 
Q 93+344  T 437  ☑ 437 
Q 6+985   T 991  ☑ 991 
Q 22+611  T 633  ☑ 633 
Q 166+395 T 561  ☑ 561 
Q 9+748   T 757  ☑ 757 
Q 733+43  T 776  ☑ 776 
Q 119+150 T 269  ☑ 269 
Q 244+211 T 455  ☑ 455 
Q 567+406 T 973  ☑ 973 
Q 248+71  T 319  ☑ 319 
Q 756+721 T 1477 ☑ 1477
Q 4+442   T 446  ☑ 446 
Q 861+89  T 950  ☑ 950 
Q 760+16  T 776  ☑ 776 
Q 131+34  T 165  ☑ 165 
Q 623+754 T 1377 ☑ 1377
Q 18+295  T 313  ☑ 313 
Q 26+463  T 489  ☑ 489 
Q 280+49  T 329  ☑ 329 
Q 798+97  T 895  ☑ 895 
Q 75+482  T 557  ☑ 557 
Q 899+54  T 953  ☑ 953 
Q 363+651 T 1014 ☑ 1014
Q 563+45  T 608 

Q 308+106 T 414  ☑ 414 
Q 267+783 T 1050 ☑ 1050
Q 117+10  T 127  ☑ 127 
Q 23+393  T 416  ☑ 416 
Q 11+393  T 404  ☑ 404 
Q 90+841  T 931  ☑ 931 
Q 813+167 T 980  ☑ 980 
Q 57+463  T 520  ☑ 520 
Q 797+789 T 1586 ☑ 1586
Q 445+374 T 819  ☑ 819 
Q 186+320 T 506  ☑ 506 
Q 175+440 T 615  ☑ 615 
Q 752+677 T 1429 ☑ 1429
Q 597+682 T 1279 ☑ 1279
Q 133+963 T 1096 ☑ 1096
Q 300+98  T 398  ☑ 398 
Q 974+729 T 1703 ☑ 1703
Q 78+539  T 617  ☑ 617 
Q 89+319  T 408  ☑ 408 
Q 93+5    T 98   ☑ 98  
Q 990+82  T 1072 ☑ 1072
Q 267+42  T 309  ☑ 309 
Q 373+82  T 455  ☑ 455 
Q 59+466  T 525  ☑ 525 
Q 432+587 T 1019 ☑ 1019
Q 798+986 T 1784 ☑ 1784
Q 24+874  T 898  ☑ 898 
Q 392+541 T 933  ☑ 933 
Q 6+437   T 443  ☑ 443 
Q 11+500  T 511  ☑ 511 
Q 312+93  T 405  ☑ 405 
Q 546+53  T 599  ☑ 599 
Q 17+230  T 247  ☑ 247 
Q 397+74  T 471  ☑ 471 
Q 221+51  T 272  ☑ 272 
Q 890+525 T 1415 ☑ 1415
Q 53+482  T 535  ☑ 535 
Q 5+342   T 347  ☑ 347 
Q 33+849  T 882  ☑ 882 
Q 254+349 T 603  ☑ 603 
Q 173+13  T 186  ☑ 186 
Q 983+130 T 1113

Q 265+221 T 486  ☑ 486 
Q 97+292  T 389  ☑ 389 
Q 77+791  T 868  ☑ 868 
Q 947+337 T 1284 ☑ 1284
Q 135+137 T 272  ☑ 272 
Q 53+72   T 125  ☑ 125 
Q 586+18  T 604  ☑ 604 
Q 433+676 T 1109 ☑ 1109
Q 854+459 T 1313 ☑ 1313
Q 911+37  T 948  ☑ 948 
Q 63+111  T 174  ☑ 174 
Q 875+881 T 1756 ☑ 1756
Q 850+78  T 928  ☑ 928 
Q 91+80   T 171  ☑ 171 
Q 77+390  T 467  ☑ 467 
Q 763+7   T 770  ☑ 770 
Q 288+61  T 349  ☑ 349 
Q 764+326 T 1090 ☑ 1090
Q 83+732  T 815  ☑ 815 
Q 25+20   T 45   ☑ 45  
Q 36+692  T 728  ☑ 728 
Q 500+31  T 531  ☑ 531 
Q 788+713 T 1501 ☑ 1501
Q 581+807 T 1388 ☑ 1388
Q 16+92   T 108  ☑ 108 
Q 61+154  T 215  ☑ 215 
Q 72+146  T 218  ☑ 218 
Q 552+927 T 1479 ☑ 1479
Q 583+610 T 1193 ☑ 1193
Q 767+18  T 785  ☑ 785 
Q 208+24  T 232  ☑ 232 
Q 50+44   T 94   ☑ 94  
Q 19+975  T 994  ☑ 994 
Q 29+37   T 66   ☑ 66  
Q 49+859  T 908  ☑ 908 
Q 568+57  T 625  ☑ 625 
Q 263+10  T 273  ☑ 273 
Q 182+57  T 239  ☑ 239 
Q 443+190 T 633  ☑ 633 
Q 306+395 T 701  ☑ 701 
Q 75+223  T 298  ☑ 298 
Q 800+159 T 959 

Q 883+1   T 884  ☑ 884 
Q 63+279  T 342  ☑ 342 
Q 83+729  T 812  ☑ 812 
Q 280+22  T 302  ☑ 302 
Q 802+851 T 1653 ☑ 1653
Q 248+990 T 1238 ☑ 1238
Q 336+82  T 418  ☑ 418 
Q 3+483   T 486  ☑ 486 
Q 716+623 T 1339 ☑ 1339
Q 230+217 T 447  ☑ 447 
Q 7+832   T 839  ☑ 839 
Q 801+242 T 1043 ☑ 1043
Q 75+936  T 1011 ☑ 1011
Q 599+997 T 1596 ☒ 1595
Q 986+68  T 1054 ☑ 1054
Q 604+86  T 690  ☑ 690 
Q 919+29  T 948  ☑ 948 
Q 956+138 T 1094 ☑ 1094
Q 641+544 T 1185 ☑ 1185
Q 992+68  T 1060 ☑ 1060
Q 39+565  T 604  ☑ 604 
Q 990+38  T 1028 ☑ 1028
Q 2+565   T 567  ☑ 567 
Q 84+793  T 877  ☑ 877 
Q 3+472   T 475  ☑ 475 
Q 42+8    T 50   ☑ 50  
Q 12+434  T 446  ☑ 446 
Q 749+66  T 815  ☑ 815 
Q 138+463 T 601  ☑ 601 
Q 695+211 T 906  ☑ 906 
Q 692+39  T 731  ☑ 731 
Q 285+53  T 338  ☑ 338 
Q 704+136 T 840  ☑ 840 
Q 530+831 T 1361 ☑ 1361
Q 214+859 T 1073 ☑ 1073
Q 51+824  T 875  ☑ 875 
Q 308+832 T 1140 ☑ 1140
Q 954+32  T 986  ☑ 986 
Q 47+402  T 449  ☑ 449 
Q 645+908 T 1553 ☑ 1553
Q 47+166  T 213  ☑ 213 
Q 349+687 T 1036

Q 89+61   T 150  ☑ 150 
Q 488+75  T 563  ☑ 563 
Q 141+442 T 583  ☑ 583 
Q 29+120  T 149  ☑ 149 
Q 808+18  T 826  ☑ 826 
Q 765+11  T 776  ☑ 776 
Q 961+64  T 1025 ☒ 1035
Q 996+8   T 1004 ☑ 1004
Q 614+28  T 642  ☑ 642 
Q 419+82  T 501  ☑ 501 
Q 93+99   T 192  ☑ 192 
Q 502+277 T 779  ☑ 779 
Q 785+30  T 815  ☑ 815 
Q 6+204   T 210  ☑ 210 
Q 734+49  T 783  ☑ 783 
Q 442+562 T 1004 ☑ 1004
Q 592+5   T 597  ☑ 597 
Q 339+746 T 1085 ☑ 1085
Q 397+862 T 1259 ☑ 1259
Q 847+6   T 853  ☑ 853 
Q 715+702 T 1417 ☑ 1417
Q 287+35  T 322  ☑ 322 
Q 713+564 T 1277 ☑ 1277
Q 693+313 T 1006 ☑ 1006
Q 32+818  T 850  ☑ 850 
Q 37+91   T 128  ☑ 128 
Q 709+137 T 846  ☑ 846 
Q 624+891 T 1515 ☑ 1515
Q 45+328  T 373  ☑ 373 
Q 6+234   T 240  ☑ 240 
Q 35+467  T 502  ☑ 502 
Q 198+54  T 252  ☑ 252 
Q 5+871   T 876  ☑ 876 
Q 27+82   T 109  ☑ 109 
Q 418+104 T 522  ☑ 522 
Q 763+573 T 1336 ☑ 1336
Q 0+356   T 356  ☑ 356 
Q 526+2   T 528  ☑ 528 
Q 946+770 T 1716 ☑ 1716
Q 485+214 T 699  ☒ 799 
Q 18+246  T 264  ☑ 264 
Q 15+199  T 214 